In [1]:
'''
1.CIFAR-10データセットの読み込み
'''
from tensorflow.keras import datasets
(x_train, t_train), (x_test, t_test) = datasets.cifar10.load_data()

In [2]:
'''
2.畳み込みネットワークの構築
''' 
from tensorflow.keras import models, layers, optimizers, regularizers

# L2正則化の係数
weight_decay = 1e-4

# CNNを構築
model = models.Sequential()

# (第1層)畳み込み層1 正則化を行う
model.add(
    layers.Conv2D(
        filters=32,                    # フィルターの数は32
        kernel_size=(3,3),             # 3×3のフィルターを使用
        input_shape=x_train.shape[1:], # 入力データの形状
        padding='same',                # ゼロパディングを行う 
        kernel_regularizer=regularizers.l2(weight_decay),
        activation='relu'              # 活性化関数はReLU
        ))

# (第2層)プーリング層1：ウィンドウサイズは2×2
model.add(
    layers.MaxPooling2D(pool_size=(2,2)))

# (第3層)畳み込み層2　正則化を行う
model.add(
    layers.Conv2D(
        filters=128,                # フィルターの数は64
        kernel_size=(3,3),          # 3×3のフィルターを使用
        padding='same',             # ゼロパディングを行う 
        kernel_regularizer=regularizers.l2(weight_decay),
        activation='relu'           # 活性化関数はReLU
        ))

# (第4層)プーリング層2：ウィンドウサイズは2×2
model.add(
    layers.MaxPooling2D(pool_size=(2,2)))

# (第5層)畳み込み層3　正則化を行う
model.add(
    layers.Conv2D(
        filters=256,                # フィルターの数は256
        kernel_size=(3,3),          # 3×3のフィルターを使用
        padding='same',             # ゼロパディングを行う 
        kernel_regularizer=regularizers.l2(weight_decay),
        activation='relu'           # 活性化関数はReLU
        ))

# (第6層)プーリング層2：ウィンドウサイズは2×2
model.add(
    layers.MaxPooling2D(pool_size=(2,2)))

# Flatten
model.add(layers.Flatten())

# ドロップアウト：ドロップアウトは40％
model.add(layers.Dropout(0.4))
 
# （第7層）全結合層
model.add(
    layers.Dense(
        512,                   # ニューロン数は512
        activation='relu'))    # 活性化関数はReLU


# （第8層）出力層
model.add(
    layers.Dense(
        10,                    # 出力層のニューロン数は10
        activation='softmax')) # 活性化関数はソフトマックス

# 学習率
learning_rate = 0.1                  

# Sequentialオブジェクトのコンパイル
model.compile(
    # 損失関数はスパースラベル対応クロスエントロピー誤差
    loss='sparse_categorical_crossentropy',
    # オプティマイザーはSGD
    optimizer=optimizers.SGD(lr=learning_rate),
    # 学習評価として正解率を指定
    metrics=['accuracy'])

# モデルのサマリを表示
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 128)       36992     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 256)         295168    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 256)         0         
_________________________________________________________________
flatten (Flatten)            (None, 4096)              0

In [ ]:
%%time
'''
3.学習の実行
'''
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# ミニバッチのサイズ
batch_size = 64

# データ拡張
datagen = ImageDataGenerator(
    rescale=1.0/255.0,      # ピクセル値を255で割って正規化する
    validation_split=0.2,   # 20パーセントのデータを検証用にする
    rotation_range=15,      # 15度の範囲でランダムに回転させる
    width_shift_range=0.1,  # 横サイズの0.1の割合でランダムに水平移動
    height_shift_range=0.1, # 縦サイズの0.1の割合でランダムに垂直移動
    horizontal_flip=True,  # 水平方向にランダムに反転、左右の入れ替え
    zoom_range=0.2,         # ランダムに拡大
)

# 訓練データ用のジェネレーターを生成
training_generator = datagen.flow(x_train, t_train,
                                  batch_size=batch_size,
                                  subset='training')      # 訓練用のデータを生成
# 検証データ用のジェネレーターを生成
validation_generator = datagen.flow(x_train, t_train,
                                    batch_size=batch_size,
                                    subset='validation') # 検証用のデータを生成
# エポック数
epochs = 100

# 学習を行う
history = model.fit(
    # 拡張データをミニバッチの数だけ生成
    training_generator,
    epochs=epochs,         # エポック数
    verbose=1,             # 学習の進捗状況を出力する
    validation_data=validation_generator,  # 20パーセントのデータを検証に使用
)

Train for 625 steps, validate for 157 steps
Epoch 1/100
625/625 [==============================] - 160s 256ms/step - loss: 2.0024 - accuracy: 0.2821 - val_loss: 1.6956 - val_accuracy: 0.3988
Epoch 2/100
625/625 [==============================] - 160s 257ms/step - loss: 1.5946 - accuracy: 0.4340 - val_loss: 1.4824 - val_accuracy: 0.4730
Epoch 3/100
625/625 [==============================] - 158s 253ms/step - loss: 1.4467 - accuracy: 0.4884 - val_loss: 1.3231 - val_accuracy: 0.5404
Epoch 4/100
625/625 [==============================] - 160s 256ms/step - loss: 1.3395 - accuracy: 0.5311 - val_loss: 1.2900 - val_accuracy: 0.5495
Epoch 5/100
625/625 [==============================] - 160s 257ms/step - loss: 1.2505 - accuracy: 0.5656 - val_loss: 1.1435 - val_accuracy: 0.6083
Epoch 6/100
625/625 [==============================] - 160s 256ms/step - loss: 1.1803 - accuracy: 0.5919 - val_loss: 1.0823 - val_accuracy: 0.6363
Epoch 7/100
625/625 [==============================] - 161s 257ms/step - l

625/625 [==============================] - 200s 320ms/step - loss: 0.5469 - accuracy: 0.8407 - val_loss: 0.7169 - val_accuracy: 0.7856
Epoch 57/100
625/625 [==============================] - 191s 305ms/step - loss: 0.5518 - accuracy: 0.8395 - val_loss: 0.7017 - val_accuracy: 0.7909
Epoch 58/100
625/625 [==============================] - 194s 310ms/step - loss: 0.5474 - accuracy: 0.8406 - val_loss: 0.7000 - val_accuracy: 0.7966
Epoch 59/100
625/625 [==============================] - 191s 306ms/step - loss: 0.5439 - accuracy: 0.8443 - val_loss: 0.6942 - val_accuracy: 0.7987
Epoch 60/100
625/625 [==============================] - 191s 305ms/step - loss: 0.5404 - accuracy: 0.8441 - val_loss: 0.6580 - val_accuracy: 0.8091
Epoch 61/100
625/625 [==============================] - 197s 315ms/step - loss: 0.5384 - accuracy: 0.8461 - val_loss: 0.6815 - val_accuracy: 0.7999
Epoch 62/100
625/625 [==============================] - 189s 302ms/step - loss: 0.5342 - accuracy: 0.8469 - val_loss: 0.6870 

In [ ]:
'''
4.損失と精度の推移をグラフにする
'''
import matplotlib.pyplot as plt
%matplotlib inline

def plot_history(history):
    # 精度の履歴をプロット
    plt.plot(history.history['accuracy'],"-",label="accuracy")
    plt.plot(history.history['val_accuracy'],"-",label="val_acc")
    plt.title('model accuracy')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.legend(loc="lower right")
    plt.show()

    # 損失の履歴をプロット
    plt.plot(history.history['loss'],"-",label="loss",)
    plt.plot(history.history['val_loss'],"-",label="val_loss")
    plt.title('model loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(loc='upper right')
    plt.show()
# modelに学習させた時の変化の様子をplot
plot_history(history)

In [ ]:
'''
5. テストデータによるモデルの評価
'''
# テストデータを正規化
x_test = x_test / 255

# 学習済みのモデルにテストデータを入力して損失と精度を取得
test_loss, test_acc = model.evaluate(x_test, t_test, verbose=0)
print('test_loss:', test_loss)
print('test_acc:', test_acc)